In [26]:
# Train 데이터    
with open('train_moong.pickle', 'rb') as fr:
    train =pickle.load(fr)
with open('train_label_moong.pickle', 'rb') as fr:
    y_train =pickle.load(fr)

# Test 데이터    
with open('test_moong.pickle', 'rb') as fr:
    test =pickle.load(fr)
with open('test_label_moong.pickle', 'rb') as fr:
    y_test =pickle.load(fr)

In [27]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import BaggingClassifier

from sklearn.metrics import accuracy_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle 

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# tfidf 상위 단어 구해서 사전 만들기

X_train = np.array(train)

tfidfv = TfidfVectorizer(preprocessor=' '.join, max_features=35000).fit(X_train)
score = {key : value for key, value in zip(sorted(tfidfv.vocabulary_, key = lambda x : tfidfv.vocabulary_[x]), tfidfv.transform(X_train).toarray().sum(axis=0))}
top = sorted(score, key = lambda x : score[x],reverse=True)[:30000]

X_train = list(map(' '.join, train))
X_test = list(map(' '.join, test))

# Train set fit, transform
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(top)
X_train = vectorizer.transform(X_train).toarray()

# Test set transform
X_test = vectorizer.transform(X_test).toarray()

nbc = MultinomialNB()
nbc.fit(X_train, y_train)
y_pred = nbc.predict(X_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_test, y_pred)))

In [15]:
X_train.shape

(146182, 23912)

In [163]:
# train으로 w2v 학습시키기
from gensim.models import Word2Vec
model = Word2Vec(train, size=128, window=3, min_count=1, iter=100, sg=1)

In [298]:
# 유사한 단어 2개 리턴
def getSimilar(word):
    return [k for k,v in model.wv.most_similar(word)

In [370]:
words = top

#유사 단어 단어 사전에 넣기
sim = []
for pw in tqdm(words):
    sim.extend(getSimilar(pw))
words.extend(sim)

X_train = list(map(' '.join, train))
X_test = list(map(' '.join, test))

# Train set fit, transform
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(words)
X_train = vectorizer.transform(X_train).toarray()

# Test set transform
X_test = vectorizer.transform(X_test).toarray()

In [371]:
print(X_train.shape)
print(X_test.shape)

(146182, 17089)
(49157, 17089)


In [372]:
nbc = MultinomialNB()
nbc.fit(X_train, y_train)
y_pred = nbc.predict(X_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_test, y_pred)))

테스트 정확도: 0.827


In [366]:
# 0.82
x11 = X_train
x12 = X_test

# 0.806
x21 = X_train
x22 = X_test

# 0.794
x31 = X_train
x32 = X_test

# 0.824
# tfidf 20000
x41 = X_train
x42 = X_test

# 0.825
# tfidf 24000
x51 = X_train
x52 = X_test

# 0.823
# tf + kai + sim
x61 = X_train
x62 = X_test

# 0.827
# tfidf 24000 / new2
x71 = X_train
x72 = X_test

In [332]:
x51.shape

(145791, 15739)

In [244]:
joblib.dump(X_train,'X_train.sav')  

['X_train.sav']

In [245]:
joblib.dump(X_test,'X_test.sav')  

['X_test.sav']

In [3]:
# 10000개 sample 준비

with open('X_sample.pickle', 'rb') as fr:
    X_sample = pickle.load(fr)
    
with open('Y_sample.pickle', 'rb') as fr:
    Y_sample = pickle.load(fr)

In [4]:
# Sample 자르기 (train / test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sample,Y_sample,random_state=2020)

In [ ]:
# 카이제곱 

In [8]:
X_train = np.array(train)

def wordCount(data, w):
    cnt = 0;
    for s in data:
        if w in s:
            cnt += 1
    return cnt

def getKai(data, w):
    A = wordCount(data[0], w)
    B = wordCount(data[1], w)
    C = len(data[0]) - A
    D = len(data[1]) - B
    # 분모가 0이면 0을 리턴함.
    if ((A+B)*(A+C)*(B+D)*(C+D)) == 0:
        return 0
    return ((A+B+C+D)*((A*D-B*C)*(A*D-B*C))) / ((A+B)*(A+C)*(B+D)*(C+D))

import itertools
words = list(itertools.chain.from_iterable(train))
words = list(set(words))

data = [X_train[y_train == 0].tolist(), X_train[y_train == 1].tolist()]

kai_dict = []
for w in tqdm(words):
    kai_dict.append(getKai(data, w))

    kaiRate = pd.DataFrame()
kaiRate['words'] = words
kaiRate['word_kai'] = kai_dict

kaiRate = kaiRate.sort_values(by=['word_kai'],ascending=False)

100%|████████████████████████████████████████████████████████████████████████████| 40480/40480 [20:56<00:00, 32.22it/s]


NameError: name 'kaiRatep' is not defined

In [49]:
from gensim.models import Word2Vec
model2 = Word2Vec(train, size=128, window=3, min_count=1, iter=100, sg=1)

In [82]:
def getSimilar(word):
    return [k for k,v in model2.wv.most_similar(word)[:2]]

In [25]:
topkai = kaiRate['words'][:30000].tolist()

NameError: name 'kaiRate' is not defined

In [24]:
topkai

NameError: name 'topkai' is not defined

In [83]:
kai = topkai

sim = []
for pw in tqdm(kai):
    sim.extend(getSimilar(pw))
kai.extend(sim)

X_train = list(map(' '.join, train))
X_test = list(map(' '.join, test))

vectorizer = CountVectorizer()
vectorizer.fit(kai)

X_train = vectorizer.transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()


100%|███████████████████████████████████████████████████████████████████████████| 13000/13000 [00:18<00:00, 705.51it/s]


In [84]:
print(X_train.shape)
print(X_test.shape)

(145791, 25096)
(48995, 25096)


In [112]:
import joblib


In [113]:

filename = 'X_train.sav'


['X_train.sav']

In [141]:
with open('sparse.pickle', 'wb') as fr:
    pickle.dump(sparse_mat, fr)

In [140]:
import numpy as np
from scipy import sparse
from scipy.sparse import csr_matrix

sparse_mat = sparse.csr_matrix(X_train)

In [85]:
nbc = MultinomialNB()
nbc.fit(X_train, y_train)
y_pred = nbc.predict(X_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_test, y_pred)))

테스트 정확도: 0.812
